In [ ]:
import pandas as pd
import duckdb

In [ ]:
con = duckdb.connect("dados_duckdb.db", read_only=False)

In [ ]:
df = con.execute("""
    SELECT *
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
        FROM bronze_z0019
        WHERE data_ingestao >= '2025-01-11'
    )
    WHERE row = 1
""").fetchdf()

df.head(10)

In [ ]:
# Passo 1: Remoção das colunas (mantido da primeira linha do código original)
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])

# Passo 2: Renomear todas as colunas de uma vez
df_final = df_final.rename(columns={
    "NATBR": "id",
    "MAKTX": "nm_produto",
    "WERKS": "id_categoria",
    "MAINS": "id_fornecedor",
    "LABST": "vl_preco"
})

# Passo 3: Visualizar o resultado (mantido da última linha do código original)
df_final.head(10)

In [ ]:
df2 = df_final.copy()
df2 = df2.astype({
    'id': int,
    'nm_produto': str,
    'id_categoria': str,
    'id_fornecedor': int,
    'vl_preco': float
})

df2.dtypes
#df2.head(10)

In [ ]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos (
    id BIGINT,
    nm_produto TEXT,
    id_categoria TEXT,
    id_fornecedor BIGINT,
    vl_preco FLOAT
)
""")

In [ ]:
con.execute("INSERT INTO produtos SELECT * from df2")

df_resultado = con.execute("select * from produtos").fetchdf()
df_resultado.head(10)

In [ ]:
con.close()